In [1]:
from datasets import load_dataset

cv_16_train = load_dataset("mozilla-foundation/common_voice_16_1", "pt", split="train")

In [2]:
cv_16_train

In [18]:
import os

from tqdm import tqdm
import librosa
import soundfile as sf
import json

In [15]:
data_folder = 'data'
wav_folder = os.path.join(data_folder, 'wav_data')
manifest_folder = os.path.join(data_folder, 'manifest_data')

In [21]:
def convert_hf_dataset_to_wav(dataset, dataset_type, manifest_filename):

    manifest_filepath = os.path.join(manifest_folder, manifest_filename)

    with open(manifest_filepath, 'w') as manifest_f:
        for sample in tqdm(dataset, desc="Converting HF Dataset to Manifest: "):
            if dataset_type == 'mls':
                text_column = 'text'
            elif dataset_type == 'common_voice':
                text_column = 'sentence'
            elif dataset_type == 'customized':
                text_column = 'text'
            else:
                raise ValueError("Dataset type not supported")

            transcription = sample[text_column]
            audio = sample['audio']
            audio_name = os.path.splitext(os.path.basename(audio['path']))[0]
            wav_file_path = os.path.join(wav_folder, audio_name + '.wav')

            if not os.path.exists(wav_file_path):
                try:
                    # Resample to 16kHz
                    audio_resampled = librosa.resample(y=audio['array'], orig_sr=audio['sampling_rate'], target_sr=16000)
                    sf.write(wav_file_path, audio_resampled, samplerate=16000)
                    duration = librosa.get_duration(y=audio_resampled, sr=16000)
                except sf.LibsndfileError as e:
                    print("Error:", e, "with audio:", audio_name)
                    continue
            else:
                duration = librosa.get_duration(filename=wav_file_path)

            manifest_line = {
                'audio_filepath': wav_file_path,
                'text': transcription,
                'duration': duration,
            }

            json.dump(manifest_line, manifest_f, ensure_ascii=False)
            manifest_f.write('\n')

In [22]:
convert_hf_dataset_to_wav(dataset=cv_16_train, dataset_type='common_voice', manifest_filename='common_voice_16_1_train_manifest.json')

Converting HF Dataset to Manifest:   0%|          | 0/21685 [00:00<?, ?it/s]/var/folders/mp/1mlhtgp14_x4hhcdlgr2xhy40000gp/T/ipykernel_3689/4119253124.py:31: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=wav_file_path)
Converting HF Dataset to Manifest:  16%|█▌        | 3519/21685 [00:15<01:22, 220.43it/s]


KeyboardInterrupt: 

In [4]:


def process_sample(sample):
    audio = sample['audio']
    audio_name = audio['path'].split('/')[-1].split('.')[0]
    if not os.path.exists(audio_name):
        try:
            audio_resampled = librosa.resample(y=audio['array'], orig_sr=audio['sampling_rate'], target_sr=16000)
            sf.write('/quartznet_train/'+audio_name+'.wav', audio_resampled, samplerate=16000)
        except sf.LibsndfileError as e:
            print(audio_name)


# Initialize tqdm with the total number of samples
progress_bar = tqdm(total=len(dataset_train), desc="Processing samples")

# Process each sample in the dataset
for sample in dataset_train:
    process_sample(sample)
    progress_bar.update(1)

# Close the progress bar after finishing the loop
progress_bar.close()

{'client_id': 'be4164b2664c4d9cdf7d10cf624dfc0e4eb06c2284eb1448bb32273f1513af423e500234654641384a19e5c04c2edfbcd0a1b3fbcc9e296b5ffd79856efce5ca',
 'path': '/Users/tmsantos/.cache/huggingface/datasets/downloads/extracted/012e9243ba77a77e48628f0af282362b49a8d918325a3f1f8c5786507fed6b95/pt_train_0/common_voice_pt_25163425.mp3',
 'audio': {'path': '/Users/tmsantos/.cache/huggingface/datasets/downloads/extracted/012e9243ba77a77e48628f0af282362b49a8d918325a3f1f8c5786507fed6b95/pt_train_0/common_voice_pt_25163425.mp3',
  'array': array([ 4.83169060e-13, -2.27373675e-13, -3.41060513e-13, ...,
          9.39775055e-06,  6.62311663e-07, -1.66810230e-06]),
  'sampling_rate': 48000},
 'sentence': 'Vitória de Santo Antão',
 'up_votes': 2,
 'down_votes': 0,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'pt',
 'segment': '',
 'variant': ''}